# Research Project - Poacher Detection: A Machine Learning Approach
## Mahdiyah Manjra - 2140796
### Superivised BY: Prof. Ewert and Prof.Sanders
### HoG Descriptors and Linear Support Vector Classifiers

In [ ]:
# import required libraries
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from skimage import img_as_float32 as as_float
from skimage.feature import hog
import scipy.misc
import imageio
#from sklearn.externals import joblib
import random as rand
import numpy as np 
import cv2
import glob
import time

import matplotlib.pyplot as plt
%matplotlib inline

## Define different HoG descriptors

In [ ]:
def hogDesc1(image):
    features = hog(image, 
                            orientations = 9, 
                            pixels_per_cell = (32, 32),
                            cells_per_block = (2, 2),
                            multichannel=True)
    return features

In [ ]:
def hogDesc2(image):
    features = hog(image, 
                            orientations = 9, 
                            pixels_per_cell = (32,32),
                            cells_per_block = (1, 1),
                            multichannel=True)
    return features

In [ ]:
def hogDesc3(image):
    features = hog(image, 
                            orientations = 9, 
                            pixels_per_cell = (16, 16),
                            cells_per_block = (1, 1),
                            multichannel=True)
    return features

In [ ]:
def hogDesc4(image):
    features = hog(image, 
                            orientations = 9, 
                            pixels_per_cell = (32, 32),
                            cells_per_block = (8, 8),
                            multichannel=True)
    return features

# Loading and Resizing the data

In [ ]:
print("Loading images to memory...")
t_start = time.time()

human_imgs=[]
nonhuman_imgs=[]

# read in images from folders
human_paths = glob.glob('drive/My Drive/Data_images/test/*.jfif')
nonhuman_paths = glob.glob('drive/My Drive/Data_images/nonhumans/*.jfif')

#human_imgs = np.array([as_float(imageio.imread(path)) for path in human_paths])
#nonhuman_imgs = np.array([as_float(imageio.imread(path)) for path in nonhuman_paths])

for path in human_paths: human_imgs.append(imageio.imread(path))
for path in nonhuman_paths: nonhuman_imgs.append(imageio.imread(path))

# resize images
scale = 0.0625

human_imgs_rs = [cv2.resize(img, None, fx=scale, fy=scale) for img in human_imgs]
nonhuman_imgs_rs = [cv2.resize(img, None, fx=scale, fy=scale) for img in nonhuman_imgs]

# save images as array
human_imgs1 = np.array(human_imgs_rs, dtype=object)
nonhuman_imgs1 = np.array(nonhuman_imgs_rs, dtype=object)

# get total number of human and nonhuman images 
total_humans = human_imgs1.shape[0]
total_nonhumans= nonhuman_imgs1.shape[0]

print("... Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Human images shape: ", human_imgs1.shape)
print("Non-Human images shape: ", nonhuman_imgs1.shape)


Loading images to memory...
... Done
Time Taken: 21.22
Human images shape:  (26, 96, 128, 3)
Non-Human images shape:  (120, 96, 128, 3)


# Apply SVC on HoG features

In [ ]:
for i in range(1,3):

  print ("HOG descriptor: ", i)
  print("Extracting features...")
  t_start = time.time()

  human_features, nonhuman_features = [], []

  print("Humans.....")
  # apply HoG descriptors 
  for img in human_imgs:

    if (i == 1):
      features = hogDesc1(img)
    elif (i==2):
      features = hogDesc2(img)
    elif (i==3):
      features = hogDesc3(img)
    elif (i==4):
      features = hogDesc4(img)

    human_features.append(features)
    print('█', end = '')

  print()
  print("Non-Humans.....")
  # apply HoG descriptors
  for img in nonhuman_imgs:

    if (i == 1):
      features = hogDesc1(img)
    elif (i==2):
      features = hogDesc2(img)
    elif (i==3):
      features = hogDesc3(img)
    elif (i==4):
      features = hogDesc4(img)
    nonhuman_features.append(features)
    print('█', end = '')

  # save features as arrays                     
  human_features = np.asarray(human_features)
  nonhuman_features = np.asarray(nonhuman_features)

  print()
  print("...Done")
  print("Time Taken:", np.round(time.time() - t_start, 2))
  print("Human features shape: ", human_features.shape)
  print("Non-humans features shape: ", nonhuman_features.shape)


  print("Scaling features...")
  t_start = time.time()

  # standardise the data 
  unscaled_x = np.vstack((human_features, nonhuman_features)).astype(np.float64)
  scaler = StandardScaler().fit(unscaled_x)
  x = scaler.transform(unscaled_x)
  y = np.hstack((np.ones(total_humans), np.zeros(total_nonhumans)))

  print("...Done")
  print("Time Taken:", np.round(time.time() - t_start, 2))
  print(" x shape: ", x.shape, " y shape: ", y.shape)


  # Training the classifier
  print("Training classifier...")
  t_start = time.time()

  # split the data (80% train, 20% test)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = rand.randint(1, 100))

  # apply and fit Support Vector Classifier
  svc = LinearSVC()
  svc.fit(x_train, y_train)

  # get performance metrics
  accuracy = svc.score(x_test, y_test)
  #cv_scores = cross_val_score(svc, x_train, y_train, cv=5)
  y_pred = svc.predict(x_test)
  cm = confusion_matrix(y_test, y_pred)
  cr = classification_report(y_test, y_pred)

  print("...Done")
  print("Time Taken:", np.round(time.time() - t_start, 2))
  print("Accuracy: ", np.round(accuracy, 4))
  #print("CV average score: %.2f" % cv_scores.mean())
  print(cm)
  print(cr)

HOG descriptor:  1
Extracting features...
Humans.....
██████████████████████████
Non-Humans.....
████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
...Done
Time Taken: 184.5
Human features shape:  (26, 106596)
Non-humans features shape:  (120, 106596)
Scaling features...
...Done
Time Taken: 0.34
 x shape:  (146, 106596)  y shape:  (146,)
Training classifier...
...Done
Time Taken: 3.32
Accuracy:  0.7115
[[27 15]
 [ 0 10]]
              precision    recall  f1-score   support

         0.0       1.00      0.64      0.78        42
         1.0       0.40      1.00      0.57        10

    accuracy                           0.71        52
   macro avg       0.70      0.82      0.68        52
weighted avg       0.88      0.71      0.74        52

HOG descriptor:  2
Extracting features...
Humans.....
██████████████████████████
Non-Humans.....
█████████████████████████████████████████████████████████████████████████████████

In [ ]:
for i in range(3,5):

  print ("HOG descriptor: ", i)
  print("Extracting features...")
  t_start = time.time()

  human_features, nonhuman_features = [], []

  print("Humans.....")
  # apply HoG descriptors 
  for img in human_imgs:

    if (i == 1):
      features = hogDesc1(img)
    elif (i==2):
      features = hogDesc2(img)
    elif (i==3):
      features = hogDesc3(img)
    elif (i==4):
      features = hogDesc4(img)

    human_features.append(features)
    print('█', end = '')

  print()
  print("Non-Humans.....")
  # apply HoG descriptors
  for img in nonhuman_imgs:

    if (i == 1):
      features = hogDesc1(img)
    elif (i==2):
      features = hogDesc2(img)
    elif (i==3):
      features = hogDesc3(img)
    elif (i==4):
      features = hogDesc4(img)
    nonhuman_features.append(features)
    print('█', end = '')

  # save features as arrays                     
  human_features = np.asarray(human_features)
  nonhuman_features = np.asarray(nonhuman_features)

  print()
  print("...Done")
  print("Time Taken:", np.round(time.time() - t_start, 2))
  print("Human features shape: ", human_features.shape)
  print("Non-humans features shape: ", nonhuman_features.shape)


  print("Scaling features...")
  t_start = time.time()

  # standardise the data 
  unscaled_x = np.vstack((human_features, nonhuman_features)).astype(np.float64)
  scaler = StandardScaler().fit(unscaled_x)
  x = scaler.transform(unscaled_x)
  y = np.hstack((np.ones(total_humans), np.zeros(total_nonhumans)))

  print("...Done")
  print("Time Taken:", np.round(time.time() - t_start, 2))
  print(" x shape: ", x.shape, " y shape: ", y.shape)


  # Training the classifier
  print("Training classifier...")
  t_start = time.time()

  # split the data (80% train, 20% test)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = rand.randint(1, 100))

  # apply and fit Support Vector Classifier
  svc = LinearSVC()
  svc.fit(x_train, y_train)

  # get performance metrics
  accuracy = svc.score(x_test, y_test)
  #cv_scores = cross_val_score(svc, x_train, y_train, cv=5)
  y_pred = svc.predict(x_test)
  cm = confusion_matrix(y_test, y_pred)
  cr = classification_report(y_test, y_pred)

  print("...Done")
  print("Time Taken:", np.round(time.time() - t_start, 2))
  print("Accuracy: ", np.round(accuracy, 4))
  #print("CV average score: %.2f" % cv_scores.mean())
  print(cm)
  print(cr)

HOG descriptor:  3
Extracting features...
Humans.....
██████████████████████████
Non-Humans.....
████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
...Done
Time Taken: 212.92
Human features shape:  (26, 110592)
Non-humans features shape:  (120, 110592)
Scaling features...
...Done
Time Taken: 0.45
 x shape:  (146, 110592)  y shape:  (146,)
Training classifier...
...Done
Time Taken: 1.36
Accuracy:  0.5962
[[20 21]
 [ 0 11]]
              precision    recall  f1-score   support

         0.0       1.00      0.49      0.66        41
         1.0       0.34      1.00      0.51        11

    accuracy                           0.60        52
   macro avg       0.67      0.74      0.58        52
weighted avg       0.86      0.60      0.63        52

HOG descriptor:  4
Extracting features...
Humans.....
██████████████████████████
Non-Humans.....
████████████████████████████████████████████████████████████████████████████████